![](https://images.pexels.com/photos/720620/pexels-photo-720620.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=650&w=940)

# IMU Sensor

Kevin J. Walchko, Phd

---

An inertial sensor is composed of several individual sensors. Generally, at a minimum there is an accelerometer to measure inertial forces and a gyroscope to measure rotational rate. However, magnometers capable of measuring the Earth's magnetic field have become cheap and are often included in low cost IMUs. These sensors can be used almost as is to calculate useful things.

## Accelerometer

You can calculate the roll and pitch angle based on the gravity gradient.

$$
f^b = R^b_n \begin{bmatrix}
0 \\
0 \\
g 
\end{bmatrix} = 
\begin{bmatrix}
g s\theta \\
-g c\theta s\phi \\
-g c\theta c\phi
\end{bmatrix}
$$

where $f$ is the accelerometer reading in body space.

$$
\phi = arctan2(-f_y, -f_z) \\
\theta = arctan2(f_x, \sqrt{f^2_y, f^2_z})
$$

```python
def accelLevel(accelRead):
    # Takes accelerometer specific force reading when stationary and estimates
    # phi and theta from the gravity vector
    fx_b, fy_b, fz_b = accelRead
    phi_est = arctan2(-fy_b,-fz_b)
    theta_est = arctan2(fx_b, sqrt(fy_b*fy_b + fz_b*fz_b))
    return phi_est, theta_est
```

### Acceleration Errors

The acceleromenter has issues determining the level orientation when it is subjected to external accelerations (Ex: quadcopter in flight) because it is no longer just the gravity vector being measured. More complex equations need to be developed and typically used with a Kalman Filter to understand what is going on.

## Magnetometer

$$
m^L = 
\begin{bmatrix}
c\theta & s\phi s\theta & c\phi s\theta \\
0 & c\phi & -s \phi \\
-s\theta & c\theta s\phi & c\phi c\theta
\end{bmatrix} m^b
$$

where $m^b$ is the reading from the magnetometer. We can convert this to a heading reading by:

$$
\phi_{mag} = arctan2(-m^L_y,m^L_x) \\
\phi_{true} = \phi_{mag} + \alpha
$$

where $\alpha$ is the declination heading correction of the Earth's magnetic field.

```python
def magHdg(magRead, phi, theta):
    # Takes magnetometer reading and estimated phi and theta Euler angles,
    # and calculates a measured psi Euler angle (magnetic heading)
    mx_b, my_b, mz_b = magRead
    
    # Rotate by phi and theta estimates to Level (L) frame
    mx_L = mx_b*cos(theta) + my_b*sin(theta)*sin(phi) + mz_b*sin(theta)*cos(phi)
    my_L = my_b*cos(phi) - mz_b*sin(phi)
    # Calculate psi
    psi_meas = arctan2(-my_L,mx_L)
    return psi_meas
```

### Magnetometer Errors

Magnetometers are influenced by external magnetic fields from motors, buck converts, etc. They also are influenced by accelerations from rapid movements.

# References

- [IMU equations](https://charlestytler.com/measuring-quadcopter-attitude-sensors/)